## Data Preprocessing

The purpose of this notebook is to preprocess the datasets, i.e.
* create subsets with data of interest,
* deal with missing values,
* combine both datasets,
* create datasets to be used for statistical analyses and modelling.

The datasets to be analyzed are
* the Happy Planet Index for 2016 (see https://happyplanetindex.org/),
* the World Development Indicators (1960 - 2019) by the World Bank (see https://datacatalog.worldbank.org/dataset/world-development-indicators)
    

### Import standard libraries and datasets

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Import WDI data (World Development Indicators)
df_wdi = pd.read_csv('../data/raw_data/WDIData.csv')

In [3]:
# Import HPI dataset (Happy Planet Index)
df_hpi = pd.read_excel('../data/raw_data/hpi-data-2016.xlsx', sheet_name = 'Complete HPI data', header = 5, usecols = 'B:O')

In [4]:
### Data Wrangling

In [5]:
# Create DataFrame with WDI data for one year
# (in this case 2016, to be congruent with the HPI data)
year = '2016'
custom_list = [[df_wdi['Indicator Code'][row],df_wdi['Country Name'][row],df_wdi[year][row]] for row in range(len(df_wdi))]

df_columns = ['Indicator Code', 'Country Name', year]
custom_wdi_df = pd.DataFrame(custom_list, columns = df_columns)

In [6]:
print(custom_wdi_df.head())
print('\nShape:',custom_wdi_df.shape)

      Indicator Code Country Name       2016
0      PA.NUS.PPP.05   Arab World        NaN
1  PA.NUS.PRVT.PP.05   Arab World        NaN
2     EG.CFT.ACCS.ZS   Arab World  84.510171
3     EG.ELC.ACCS.ZS   Arab World  89.678685
4  EG.ELC.ACCS.RU.ZS   Arab World  79.665635

Shape: (377256, 3)


In [7]:
# Add Happy Planet Index value per Country from the HPI dataset

# Prepare HPI DataFrame for concatenation
# Include only relevant columns
df_hpi_mod = df_hpi[['Country', 'Happy Planet Index']]

# Include only relevant rows
df_hpi_mod = df_hpi_mod.iloc[:140]    

#df_hpi_mod

In [8]:
# Find out which Country Names in both datasets match
custom_wdi_df_match = custom_wdi_df.merge(df_hpi_mod, left_on='Country Name', right_on='Country', how='inner', suffixes=('WDI ','HPI '))
matching_countries_num = custom_wdi_df_match['Country Name'].unique().shape[0]
matching_countries_num

125

In [9]:
# Find out which Country Names in both datasets do not match
custom_wdi_df_all = custom_wdi_df.merge(df_hpi_mod, left_on='Country Name', right_on='Country', how='outer', suffixes=('WDI ','HPI '))
non_matching_countries_wdi = custom_wdi_df_all['Country Name'][custom_wdi_df_all['Country Name'] != custom_wdi_df_all['Country']]
non_matching_countries_wdi = non_matching_countries_wdi[non_matching_countries_wdi.isnull() == False].unique()
print('WDI not matching countries:',non_matching_countries_wdi.shape[0])

non_matching_countries_hpi = custom_wdi_df_all['Country'][custom_wdi_df_all['Country'] != custom_wdi_df_all['Country Name']]
non_matching_countries_hpi = non_matching_countries_hpi[non_matching_countries_hpi.isnull() == False].unique()
print('HPI not matching countries:',non_matching_countries_hpi.shape[0])

print(non_matching_countries_wdi)
print(non_matching_countries_hpi)

WDI not matching countries: 139
HPI not matching countries: 15
['Arab World' 'Caribbean small states' 'Central Europe and the Baltics'
 'Early-demographic dividend' 'East Asia & Pacific'
 'East Asia & Pacific (excluding high income)'
 'East Asia & Pacific (IDA & IBRD countries)' 'Euro area'
 'Europe & Central Asia' 'Europe & Central Asia (excluding high income)'
 'Europe & Central Asia (IDA & IBRD countries)' 'European Union'
 'Fragile and conflict affected situations'
 'Heavily indebted poor countries (HIPC)' 'High income' 'IBRD only'
 'IDA & IBRD total' 'IDA blend' 'IDA only' 'IDA total'
 'Late-demographic dividend' 'Latin America & Caribbean'
 'Latin America & Caribbean (excluding high income)'
 'Latin America & the Caribbean (IDA & IBRD countries)'
 'Least developed countries: UN classification' 'Low & middle income'
 'Low income' 'Lower middle income' 'Middle East & North Africa'
 'Middle East & North Africa (excluding high income)'
 'Middle East & North Africa (IDA & IBRD countri

In [10]:
# Manually create a matching table of non-matching country names
non_matching_countries_hpi
wdi_pair_list = ['Egypt, Arab Rep.',
                 'Hong Kong SAR, China',
                 'Iran, Islamic Rep.',
                 'Kyrgyz Republic',
                 'North Macedonia',
                 'Palestine',   # no match found in WDI
                 'Congo, Rep.',
                 'Russian Federation',
                 'Slovak Republic',
                 'Korea, Rep.',
                 'Eswatini',
                 'Syrian Arab Republic',
                 'United States',
                 'Venezuela, RB',
                 'Yemen, Rep.'
                ]
wdi_pair_list

['Egypt, Arab Rep.',
 'Hong Kong SAR, China',
 'Iran, Islamic Rep.',
 'Kyrgyz Republic',
 'North Macedonia',
 'Palestine',
 'Congo, Rep.',
 'Russian Federation',
 'Slovak Republic',
 'Korea, Rep.',
 'Eswatini',
 'Syrian Arab Republic',
 'United States',
 'Venezuela, RB',
 'Yemen, Rep.']

In [11]:
# Transform Indicator rows into Indicator columns
custom_wdi_df_indcols = custom_wdi_df.pivot(index = 'Country Name', columns = 'Indicator Code', values = '2016')
custom_wdi_df_indcols = custom_wdi_df_indcols.rename_axis(None, axis=1).reset_index()
custom_wdi_df_indcols.head()

,Country Name,AG.AGR.TRAC.NO,AG.CON.FERT.PT.ZS,AG.CON.FERT.ZS,AG.LND.AGRI.K2,AG.LND.AGRI.ZS,AG.LND.ARBL.HA,AG.LND.ARBL.HA.PC,AG.LND.ARBL.ZS,AG.LND.CREL.HA,...,per_sa_allsa.cov_q4_tot,per_sa_allsa.cov_q5_tot,per_si_allsi.adq_pop_tot,per_si_allsi.ben_q1_tot,per_si_allsi.cov_pop_tot,per_si_allsi.cov_q1_tot,per_si_allsi.cov_q2_tot,per_si_allsi.cov_q3_tot,per_si_allsi.cov_q4_tot,per_si_allsi.cov_q5_tot
0,Afghanistan,NaN,590.067055,12.182300,379100.000000,58.067580,7.729000e+06,0.218437,11.838679,2793133.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Albania,NaN,NaN,126.138483,11816.999512,43.127735,6.203000e+05,0.215674,22.638686,148084.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Algeria,NaN,8.209376,22.315273,413601.992188,17.365539,7.404200e+06,0.182588,3.108736,3376331.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,American Samoa,NaN,NaN,NaN,49.000001,24.500000,3.000000e+03,0.053820,15.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Andorra,NaN,NaN,NaN,187.800007,39.957448,7.800000e+02,0.010091,1.659574,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Replace non-matching country names before
df_hpi_mod2 = df_hpi_mod.replace(non_matching_countries_hpi,wdi_pair_list)

In [13]:
# Merge DataFrames
custom_wdi_df_match2 = custom_wdi_df_indcols.merge(df_hpi_mod2, left_on='Country Name', right_on='Country', how='inner')
matching_countries_num2 = custom_wdi_df_match2['Country Name'].unique().shape[0]
matching_countries_num2

139

In [14]:
# Remove columns with only missing data and duplicated Country column
nan_columns = set(custom_wdi_df_match2.columns[custom_wdi_df_match2.isnull().mean() == 1])
nan_columns

wdi_hpi_2016_df = custom_wdi_df_match2.drop(nan_columns, axis = 1)
wdi_hpi_2016_df = wdi_hpi_2016_df.drop('Country', axis = 1)

In [15]:
# Show columns without missing data
columns_no_nulls = set(wdi_hpi_2016_df.columns[wdi_hpi_2016_df.isnull().mean()==0])
columns_no_nulls

{'AG.LND.AGRI.K2',
 'AG.LND.AGRI.ZS',
 'AG.LND.ARBL.HA',
 'AG.LND.ARBL.HA.PC',
 'AG.LND.ARBL.ZS',
 'AG.LND.TOTL.K2',
 'AG.PRD.CREL.MT',
 'AG.PRD.CROP.XD',
 'AG.PRD.FOOD.XD',
 'AG.PRD.LVSK.XD',
 'AG.SRF.TOTL.K2',
 'Country Name',
 'EG.ELC.ACCS.RU.ZS',
 'EG.ELC.ACCS.UR.ZS',
 'EG.ELC.ACCS.ZS',
 'EN.POP.DNST',
 'ER.FSH.CAPT.MT',
 'ER.FSH.PROD.MT',
 'ER.LND.PTLD.ZS',
 'ER.PTD.TOTL.ZS',
 'Happy Planet Index',
 'IT.CEL.SETS',
 'IT.CEL.SETS.P2',
 'IT.MLT.MAIN',
 'IT.MLT.MAIN.P2',
 'IT.NET.SECR',
 'IT.NET.SECR.P6',
 'IT.NET.USER.ZS',
 'NY.ADJ.DMIN.CD',
 'NY.ADJ.DNGY.CD',
 'SH.H2O.BASW.ZS',
 'SH.STA.BASS.ZS',
 'SH.TBS.DTEC.ZS',
 'SH.TBS.INCD',
 'SL.AGR.EMPL.FE.ZS',
 'SL.AGR.EMPL.MA.ZS',
 'SL.AGR.EMPL.ZS',
 'SL.EMP.1524.SP.FE.ZS',
 'SL.EMP.1524.SP.MA.ZS',
 'SL.EMP.1524.SP.ZS',
 'SL.EMP.MPYR.FE.ZS',
 'SL.EMP.MPYR.MA.ZS',
 'SL.EMP.MPYR.ZS',
 'SL.EMP.SELF.FE.ZS',
 'SL.EMP.SELF.MA.ZS',
 'SL.EMP.SELF.ZS',
 'SL.EMP.TOTL.SP.FE.ZS',
 'SL.EMP.TOTL.SP.MA.ZS',
 'SL.EMP.TOTL.SP.ZS',
 'SL.EMP.VULN.FE.ZS',
 'S

In [16]:
# Evaluate columns with most missing data
columns_90_nulls = set(wdi_hpi_2016_df.columns[wdi_hpi_2016_df.isnull().mean() > 0.90])
columns_75_nulls = set(wdi_hpi_2016_df.columns[wdi_hpi_2016_df.isnull().mean() > 0.75])    # more than 75% of data is missing
columns_50_nulls = set(wdi_hpi_2016_df.columns[wdi_hpi_2016_df.isnull().mean() > 0.50])

print(len(columns_90_nulls))
print(len(columns_75_nulls))
print(len(columns_50_nulls))
print(wdi_hpi_2016_df.shape[1])

99
192
330
1274


In [17]:
# Remove columns with more than 50% of missing data
wdi_hpi_2016_df = wdi_hpi_2016_df.drop(wdi_hpi_2016_df.columns[wdi_hpi_2016_df.isnull().mean() > 0.50], axis = 1)
print(wdi_hpi_2016_df.shape)

(139, 944)


In [18]:
# Show remaining columns in order to choose columns for the model
col_descr = pd.DataFrame(wdi_hpi_2016_df.columns, columns = ['Indicator Code'])

df_wdi_inds = pd.read_csv('../data/raw_data/WDISeries.csv')

col_descr = col_descr.merge(df_wdi_inds, left_on='Indicator Code', right_on='Series Code', how='left')
col_descr.to_csv('../data/available_columns.csv')
col_descr.head()

,Indicator Code,Series Code,Topic,Indicator Name,Short definition,Long definition,Unit of measure,Periodicity,Base Period,Other notes,...,Notes from original source,General comments,Source,Statistical concept and methodology,Development relevance,Related source links,Other web links,Related indicators,License Type,Unnamed: 20
0,Country Name,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AG.CON.FERT.ZS,AG.CON.FERT.ZS,Environment: Agricultural production,Fertilizer consumption (kilograms per hectare ...,NaN,Fertilizer consumption measures the quantity o...,NaN,Annual,NaN,NaN,...,NaN,NaN,"Food and Agriculture Organization, electronic ...",Fertilizer consumption measures the quantity o...,"Factors such as the green revolution, has led ...",NaN,NaN,NaN,CC BY-4.0,NaN
2,AG.LND.AGRI.K2,AG.LND.AGRI.K2,Environment: Land use,Agricultural land (sq. km),NaN,Agricultural land refers to the share of land ...,NaN,Annual,NaN,NaN,...,NaN,NaN,"Food and Agriculture Organization, electronic ...",Agricultural land constitutes only a part of a...,Agricultural land covers more than one-third o...,NaN,NaN,NaN,CC BY-4.0,NaN
3,AG.LND.AGRI.ZS,AG.LND.AGRI.ZS,Environment: Land use,Agricultural land (% of land area),NaN,Agricultural land refers to the share of land ...,NaN,Annual,NaN,NaN,...,NaN,NaN,"Food and Agriculture Organization, electronic ...",Agriculture is still a major sector in many ec...,Agricultural land covers more than one-third o...,NaN,NaN,NaN,CC BY-4.0,NaN
4,AG.LND.ARBL.HA,AG.LND.ARBL.HA,Environment: Land use,Arable land (hectares),NaN,Arable land (in hectares) includes land define...,NaN,Annual,NaN,NaN,...,NaN,NaN,"Food and Agriculture Organization, electronic ...",Temporary fallow land refers to land left fall...,Agricultural land covers more than one-third o...,NaN,NaN,NaN,CC BY-4.0,NaN


In [19]:
# Load manually chosen columns and description
chosen_columns = pd.read_csv('../data/WDI_indicator_choice.csv')
chosen_columns = chosen_columns[['Indicator Code', 'Indicator Name', 'Manual Group']][chosen_columns['Manual Choice'] == 1]
chosen_columns = chosen_columns.reset_index().drop('index', axis = 1)
chosen_columns.to_pickle('../data/WDI_chosen_columns.pkl')
print('No. of chosen indicators:', len(chosen_columns))
chosen_columns

No. of chosen indicators: 42


,Indicator Code,Indicator Name,Manual Group
0,AG.LND.AGRI.ZS,Agricultural land (% of land area),environment
1,AG.LND.FRST.ZS,Forest area (% of land area),environment
2,AG.PRD.FOOD.XD,Food production index (2004-2006 = 100),environment
3,AG.PRD.LVSK.XD,Livestock production index (2004-2006 = 100),environment
4,EG.ELC.ACCS.ZS,Access to electricity (% of population),infrastructure
5,EN.POP.DNST,Population density (people per sq. km of land ...,environment
6,ER.PTD.TOTL.ZS,Terrestrial and marine protected areas (% of t...,environment
7,FB.ATM.TOTL.P5,"Automated teller machines (ATMs) (per 100,000 ...",infrastructure
8,FB.CBK.BRCH.P5,"Commercial bank branches (per 100,000 adults)",infrastructure
9,FP.CPI.TOTL,Consumer price index (2010 = 100),infrastructure


In [20]:
# Reduce DataFrame to chosen columns
keep_columns = pd.DataFrame(['Country Name', 'Happy Planet Index'])
keep_columns = pd.concat([keep_columns, chosen_columns['Indicator Code']], ignore_index = True).rename(columns = {0:'Indicator Code'})
keep_columns = keep_columns['Indicator Code'].tolist()

def drop_columns(df, keep_cols):
    '''
    Drop unnecessary columns from the main DataFrame
    input:  df = DataFrame to be modified
            keep_cols = list containg column names (as string) which are to be kept in the DataFrame
    output: df = modified DataFrame only containing columns from keep_cols
    '''
    drop_cols = []
    for col in df.columns:
        if not col in keep_cols:
            drop_cols.append(col)    
    df = df.drop(drop_cols, axis = 1)
    return df
    
wdi_hpi_2016_df = drop_columns(wdi_hpi_2016_df, keep_columns)
wdi_hpi_2016_df

,Country Name,AG.LND.AGRI.ZS,AG.LND.FRST.ZS,AG.PRD.FOOD.XD,AG.PRD.LVSK.XD,EG.ELC.ACCS.ZS,EN.POP.DNST,ER.PTD.TOTL.ZS,FB.ATM.TOTL.P5,FB.CBK.BRCH.P5,...,SE.TER.ENRR,SE.XPD.TOTL.GB.ZS,SE.XPD.TOTL.GD.ZS,SH.H2O.BASW.ZS,SH.H2O.SMDW.ZS,SH.MED.PHYS.ZS,SH.STA.BASS.ZS,SP.DYN.LE00.IN,SP.POP.GROW,Happy Planet Index
0,Afghanistan,58.067580,2.067825,125.41,98.70,97.700000,54.197114,0.104707,1.052141,2.094214,...,NaN,16.21170,4.22836,64.286484,NaN,0.2840,42.054183,63.763000,2.778317,20.225350
1,Albania,43.127735,28.121897,150.93,113.07,100.000000,104.967190,13.147749,33.806812,21.509584,...,58.38185,13.59696,3.95464,91.020370,69.984329,1.1998,97.704877,78.194000,-0.159880,36.766874
2,Algeria,17.365539,0.824439,151.62,147.06,99.992317,17.025957,7.105500,8.680286,5.241517,...,42.62885,NaN,NaN,93.515851,NaN,1.8300,87.542316,76.298000,2.051355,33.300543
3,Argentina,54.335712,9.798406,129.11,107.00,99.966881,15.928135,7.528593,50.331821,13.447737,...,87.21345,13.37615,5.57218,99.078375,NaN,NaN,94.258505,76.221000,1.057182,35.190244
4,Armenia,58.897086,11.668423,135.37,143.85,100.000000,103.131226,23.109003,60.903516,23.008940,...,51.08019,10.19720,2.75812,99.898536,83.005976,NaN,93.406972,74.640000,0.361431,25.666417
5,Australia,48.241944,16.258278,105.58,102.02,100.000000,3.144936,28.591842,167.585527,27.722792,...,120.96570,13.76601,5.28031,99.969972,NaN,3.5874,99.991152,82.448780,1.561940,21.228966
6,Austria,32.356676,46.905713,101.29,108.27,100.000000,105.869493,28.443983,162.782099,12.644693,...,83.45495,10.95385,5.50070,100.000000,98.907808,5.1441,99.974814,81.641463,1.081396,30.478224
7,Bangladesh,70.632327,10.957978,145.31,135.88,75.920000,1213.573327,4.888676,8.045164,8.728950,...,17.87436,11.41942,1.53554,96.882530,55.491263,0.4822,47.005803,71.785000,1.091300,38.390694
8,Belarus,42.035489,42.630106,133.94,147.85,100.000000,46.808351,9.511258,55.252115,0.856622,...,88.18458,12.15878,4.94655,96.474184,94.497432,NaN,97.777751,73.826829,0.125511,21.718916
9,Belgium,44.610305,22.583885,84.89,81.32,100.000000,374.221334,24.578162,88.357865,36.557841,...,75.89064,12.33788,6.54428,99.999999,99.523184,3.3234,99.486058,81.439024,0.506300,23.726523


In [21]:
# Dealing with NaN values: Impute column mean for NaN values
wdi_hpi_2016_df = wdi_hpi_2016_df.fillna(wdi_hpi_2016_df.mean())

### Split and save datasets

In [22]:
# Save preprocessed DataFrame
wdi_hpi_2016_df.to_pickle('../data/wdi_hpi_2016_df.pkl')

In [23]:
# Separate preprocessed DataFrame into groups for individual regression models
# i. e. separate df for economic variables, environmental variables, ...
for group in chosen_columns['Manual Group'][chosen_columns['Manual Group'].isnull() == False].unique():
    file_name = '../data/wdi_hpi_2016_' + group + '.pkl'
    keep_columns = ['Country Name', 'Happy Planet Index']
    for ind in chosen_columns['Indicator Code'][chosen_columns['Manual Group'] == group]:
        keep_columns.append(ind)
    group_df = drop_columns(wdi_hpi_2016_df, keep_columns)
    group_df.to_pickle(file_name)